In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from io_images import get_images_infoframe
import paramiko
import os 
from process_ims import get_ims_pixel_size

Get chunk images .tiff

In [ ]:
windows = "nt" in os.name

In [ ]:
project_path = "results/retina_example"
infoframe = get_images_infoframe(project_path, 
                                 extension='.tif',
                                 conditions = [],
)

In [ ]:
infoframe

Get one chunk

In [ ]:
# Define the file name you are searching for
filename = infoframe['file_name'].iloc[4]
filepath = infoframe['file_path'].iloc[4]
# Get the index of the row containing the file name
index = infoframe.loc[infoframe['file_name'] == filename].index

# If you want to print or use the index
print(f"Index of the row: {index}")


get metadata

In [ ]:
# (0.9, 0.339, 0.339)
1/0.339

Get the one

In [ ]:
# base_path = f'/mnt/archive/archive/siegegrp/AlVe/MORPHOMICS2.0_MICROGLIA_BRAIN_ATLAS'
base_path = "results/retina_example"
function = f'CUDA_VISIBLE_DEVICES=0 trAIce img2swc'
linux_folderpath = f' -ip {base_path}/{filename}' 

cube_params = f' -wss "(128, 128, 16)" -wsb "(128, 128, 16)" -tp "(2.95, 2.95, 1.1)"'
save_folderpath = f' -spd results/traced_microglia/ -mp ./ -spsl ./ -nw 1 -bsp ./'


In [ ]:
command = function + linux_folderpath + cube_params + save_folderpath

In [ ]:
print(command)

In [ ]:
import subprocess
subprocess.run(command, shell=True, check=True)
